In [1]:
import pandas as pd

import statsmodels.api as sm
import pandas as pd
import datetime
from pathlib import Path

pd.set_option("display.max_columns", None)
# !pip install statsmodels


# %config InlineBackend.figure_format = "svg"
# %config InlineBackend.print_figure_kwargs = {"dpi" : 300}
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd


# from cadFace.vis import percentiles_plot
import sci_palettes

try:
    sci_palettes.register_cmap()
except:
    pass
import scienceplots
from pathlib import Path
import pandas as pd
from ppp_aging.ppp_model import *

plt.style.use(["nature", "no-latex"])
sns.set_context("paper", font_scale=1.5)
sns.set_palette("nejm")
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg')
import scienceplots

plt.style.use(["nature", "no-latex"])

/tmp/ipykernel_9266/4284081116.py:36: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('svg')


In [7]:
model_obj = {
    Path(i.name).stem: pd.read_pickle(str(i))
    for i in Path("./1_regression_models/organ").glob("*.pkl")
}
model_obj

{'Immune + RF': {'model': LogisticRegression(),
  'train_metrics': {'train_auc': 0.7790007603772653},
  'test_metrics': {'test_auc': 0.7600093446539681},
  'train_data':        incident_cad     HBQ1      CD5L    CSF2RB   SIRPB1  CLEC4D    LILRA6  \
  18140             0  0.38840  0.322400  0.378800  0.09230  0.9970 -0.466750   
  38377             0  0.77150  0.218300 -0.749800  0.25630  0.1031 -0.621550   
  17685             0 -0.53905 -0.072500  0.328200  0.33405 -0.6032  1.521950   
  11638             1  0.89750  0.015165  0.004500  0.67270  0.4830  0.999650   
  28125             0 -0.33500 -1.581700  0.820900  0.23900  0.1381 -1.192850   
  ...             ...      ...       ...       ...      ...     ...       ...   
  38543             0 -0.15740  0.014500  0.183800  0.55360  0.2448 -0.881750   
  5987              0  0.04310  0.015165 -0.033475 -0.89205 -0.7624 -0.058275   
  34612             0 -0.12310 -0.085900 -0.125000 -0.82720  0.5251  0.070550   
  25290             0 

In [6]:
combination = pd.read_pickle("organ_combination_all.pkl")
combination

OrderedDict([('Adipose', ['FABP4', 'ADIPOQ', 'LEP']),
             ('Adipose + RF',
              ['FABP4',
               'ADIPOQ',
               'LEP',
               'age',
               'sex',
               'ldl_a',
               'hdl_a',
               'tc_a',
               'tg_a',
               'apob',
               'sbp_a',
               'dbp_a']),
             ('Adipose + RF + PRS',
              ['FABP4',
               'ADIPOQ',
               'LEP',
               'age',
               'sex',
               'ldl_a',
               'hdl_a',
               'tc_a',
               'tg_a',
               'apob',
               'sbp_a',
               'dbp_a',
               'PRS']),
             ('Heart', ['BMP10', 'TNNI3', 'NPPB', 'PXDNL']),
             ('Heart + RF',
              ['BMP10',
               'TNNI3',
               'NPPB',
               'PXDNL',
               'age',
               'sex',
               'ldl_a',
               'hdl_a',
               'tc

In [14]:
test_imputed = pd.read_pickle("1_test_imputed.pkl").dropna(
    subset=combination["Adipose + RF"]
)
test_imputed

sex  height  weight       BSA  genotype_array   age      PC1  \
0      1.0   171.0    64.2  1.746282               2  44.0  71.3002   
1      0.0   165.0    55.8  1.599219               2  53.0 -12.4815   
2      1.0   171.0    77.3  1.916181               2  62.0 -11.4721   
7      0.0   171.0    73.8  1.872298               2  53.0 -13.8051   
9      0.0   177.0    71.0  1.868377               2  52.0  22.6608   
...    ...     ...     ...       ...             ...   ...      ...   
10190  1.0   194.0   111.5  2.451247               2  59.0 -11.2401   
10191  0.0   156.0    56.2  1.560555               2  47.0  89.6502   
10192  0.0   160.0    64.0  1.686548               2  61.0 -13.2795   
10193  1.0   184.0   104.1  2.306657               2  67.0 -14.3129   
10194  0.0   159.0    56.8  1.583877               2  41.0 -11.2361   

             PC2        PC3        PC4        PC5       PC6        PC7  \
0     -100.66700  81.855800   9.994410  -0.516576  4.576150   1.832860   
1        3.16181  -3.999700   0.309314  -6.865420 -1.982800  -1.245250   
2        2.20519   0.205421  -1.035540  -2.391010 -2.383070   0.886641   
7        5.46987  -4.307390   2.966630  -1.338870 -1.425630   0.249275   
9      -11.82330  28.726600 -80.967500  13.824100 -7.574850 -27.455600   
...          ...        ...        ...        ...       ...        ...   
10190    6.36919  -1.454640   0.493263  -3.298270 -2.062050  -2.349110   
10191 -130.82500  94.235400  28.885200  -3.609980 -6.661040  -4.625900   
10192    4.90337  -1.680180   4.487020   9.191780 -0.761325   0.399757   
10193    1.37825   2.037820   0.217533   1.904070 -1.357880   0.266230   
10194    4.63863  -0.627089   0.635917  -5.071890  2.164910   0.770584   

             PC8      PC9       PC10  assessment_center  age_squared      BMI  \
0       5.045020  1.61431  18.488800                  4       1936.0  21.9555   
1       1.828070 -4.04044  -0.823290                 11       2809.0  20.4959   
2      -0.934631  1.41458   1.704540                  9       3844.0  26.4355   
7      -2.754040  5.32388  -2.011550                  3       2809.0  25.2385   
9     -15.176200 -1.99456   3.787430                  7       2704.0  22.6627   
...          ...      ...        ...                ...          ...      ...   
10190  -1.497980 -2.08661   0.895716                 20       3481.0  31.5248   
10191  -6.542960 -2.72082 -13.781000                  4       2209.0  23.0934   
10192  -0.884922 -1.60040  -2.621980                  8       3721.0  25.0000   
10193   0.996555 -0.59830  -3.291160                  9       4489.0  30.7479   
10194  -1.732380 -1.51917  -4.123390                  4       1681.0  22.4675   

       dbp_a  sbp_a   tc_a   tg_a  hdl_a  ldl_a    dbp    sbp     tc     tg  \
0       76.0  115.0  5.851  0.893  1.635  3.549   76.0  115.0  5.851  0.893   
1      100.0  150.0  7.407  3.176  1.415  4.840  100.0  150.0  7.407  3.176   
2       81.0  144.0  5.909  1.869  1.670  3.587   81.0  144.0  5.909  1.869   
7       76.0  130.0  6.191  0.930  1.951  3.586   76.0  130.0  6.191  0.930   
9       60.0  106.0  5.411  0.766  1.675  3.369   60.0  106.0  5.411  0.766   
...      ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
10190   72.0  120.0  6.671  7.376  0.762  4.060   72.0  120.0  6.671  7.376   
10191   84.0  129.0  4.979  0.402  1.950  2.606   84.0  129.0  4.979  0.402   
10192   81.0  154.0  5.482  1.525  1.158  3.807   81.0  154.0  4.386  1.296   
10193   75.0  148.0  4.699  1.956  0.940  3.084   75.0  148.0  4.699  1.956   
10194   82.0  125.0  5.914  0.647  1.772  3.694   82.0  125.0  5.914  0.647   

         hdl    ldl   apob  cad  mi    crp        eGFR     Cr  CAD   cad_date  \
0      1.635  3.549  1.002    0   0   0.48  112.310853   69.9    0        NaT   
1      1.415  4.840  1.433    0   0   1.32   82.185961   74.9    0        NaT   
2      1.670  3.587  1.025    0   0   1.93   85.913239   87.7    0        NaT   
7      1.951  3.586

In [15]:

%%time
from ppp_aging.model import cal_binary_metrics_bootstrap


# method = "LR"
metrics_result = {}
for method in model_obj.keys():
    model = model_obj[method]["model"]
    if hasattr(model, "predict_proba"):
        pred_res = model.predict_proba(test_imputed[combination[method]]).iloc[:, 1]
    else:
        pred_res = model.predict(test_imputed[combination[method]].values)
    y_true = test_imputed["incident_cad"]
    metrics_result[method] = cal_binary_metrics_bootstrap(
        y_true, pred_res, ci_kwargs={"n_resamples": 100}
    )


CPU times: user 50.3 s, sys: 0 ns, total: 50.3 s
Wall time: 50.3 s


In [26]:
combination["Adipose"]

['FABP4', 'ADIPOQ', 'LEP']

In [21]:
pd.DataFrame(metrics_result).T.sort_values("AUC", ascending=False)

,AUC,AUC_UCI,AUC_LCI,ACC,ACC_UCI,ACC_LCI,Macro_F1,Macro_F1_UCI,Macro_F1_LCI,Sensitivity,Sensitivity_UCI,Sensitivity_LCI,Specificity,Specificity_UCI,Specificity_LCI,APR,APR_UCI,APR_LCI
Immune + RF + PRS,0.775382,0.793773,0.758094,0.680164,0.688485,0.670631,0.500121,0.509656,0.489873,0.756264,0.795526,0.726237,0.675913,0.685462,0.664320,0.164651,0.187707,0.133119
Liver + RF + PRS,0.773956,0.793031,0.753518,0.697276,0.708363,0.689431,0.507140,0.519542,0.496819,0.719818,0.754474,0.679360,0.696017,0.705254,0.685926,0.181328,0.204036,0.148213
Artery + RF + PRS,0.771871,0.792103,0.748710,0.678597,0.687096,0.667076,0.498012,0.508666,0.488300,0.744875,0.781818,0.695666,0.674895,0.686226,0.666284,0.173035,0.198736,0.148680
Kidney + RF + PRS,0.770900,0.794084,0.746993,0.656303,0.666730,0.647075,0.487108,0.496290,0.474228,0.774487,0.807149,0.739167,0.649701,0.661214,0.640750,0.169338,0.198310,0.141069
Lung + RF + PRS,0.769793,0.786874,0.747260,0.674741,0.683722,0.665329,0.497187,0.509480,0.486647,0.760820,0.799002,0.718822,0.669933,0.681611,0.661062,0.160213,0.188110,0.133314
Adipose + RF + PRS,0.769398,0.791624,0.750726,0.674379,0.683041,0.663527,0.497185,0.507754,0.488658,0.763098,0.796869,0.720229,0.669424,0.680920,0.659392,0.164162,0.185305,0.124578
Pancreas + RF + PRS,0.768956,0.789911,0.747834,0.705833,0.714121,0.698837,0.512087,0.524566,0.499953,0.715262,0.760939,0.675878,0.705306,0.715318,0.696264,0.156304,0.178805,0.125688
Intestine + RF + PRS,0.768170,0.791463,0.746683,0.655098,0.664964,0.646689,0.483600,0.492753,0.472277,0.744875,0.780701,0.709306,0.650083,0.659599,0.640332,0.178611,0.206340,0.139080
organismal + RF + PRS,0.768084,0.791000,0.740898,0.632803,0.641606,0.622014,0.473643,0.482481,0.465014,0.783599,0.815561,0.743371,0.624380,0.635318,0.615425,0.171696,0.196165,0.139253
Brain + RF + PRS,0.766444,0.792830,0.749583,0.667872,0.678248,0.659361,0.490503,0.499821,0.479279,0.735763,0.773017,0.700186,0.664079,0.673846,0.652708,0.178036,0.210604,0.149785
